## Nu.nl Convert Data

Convert nu.nl data to text data format

In [1]:
import datetime
import os
import pandas as pd
import re
from IPython.display import clear_output

In [2]:
DATADIRIN = "../data/nunl/comments/"
DATADIROUT = "../data/nunl/text/"
FILEPATTERN = r"."
DATEPATTERNIN = "%m/%d/%Y, %H:%M:%S %p"
DATEPATTERNOUT = "%Y%m%d-%H"
FILESUFFIX = ".out.gz"
DATE = "date"
TEXT = "text"
NAME = "name"
ID = "id"
IDSTR = "id_str"
REPLYID = "in_reply_to_status_id_str"
USER = "user"
VERIFIED = "verified"

In [ ]:
idStr = ""
replyId = ""
verified = ""
dataPerHour = {}
inFileNames = sorted(os.listdir(DATADIRIN))

for inFileName in inFileNames:
    if re.search(FILEPATTERN,inFileName):
        clear_output(wait=True)
        print(inFileName)
        try:
            dfIn = pd.read_csv(DATADIRIN+inFileName)
        except:
            continue
        for i in range(0,len(dfIn)):
            try:
                dateStringComplete = dfIn.iloc[i][DATE]
                date = datetime.datetime.strptime(dateStringComplete,DATEPATTERNIN)
                dateStringHour = date.strftime(DATEPATTERNOUT)
            except: break
            if not dateStringHour in dataPerHour: dataPerHour[dateStringHour] = []
            dataPerHour[dateStringHour].append(dfIn.iloc[i])

In [8]:
for dateString in sorted(dataPerHour.keys()):
    clear_output(wait=True)
    print(dateString)
    outFileName = dateString+FILESUFFIX
    if os.path.exists(DATADIROUT+outFileName):
        dfOut = pd.read_csv(DATADIROUT+outFileName,compression="gzip")
        for d in dataPerHour[dateString]:
            dfOut.loc[len(dfOut)] = [idStr,replyId,d[NAME],verified,d[TEXT]]
    elif len(dataPerHour[dateString]) > 0:
        dfOut = pd.DataFrame([{IDSTR:idStr,REPLYID:replyId,USER:dataPerHour[dateString][0][NAME],\
                                VERIFIED:verified,TEXT:dataPerHour[dateString][0][TEXT]}])
        for d in dataPerHour[dateString][1:]:
            dfOut.loc[len(dfOut)] = [idStr,replyId,d[NAME],verified,d[TEXT]]
    dfOut.to_csv(DATADIROUT+outFileName,index=False,compression="gzip")

20200726-11
